In [1]:
import pandas as pd

In [2]:
import os

# Load environment variables
from dotenv import load_dotenv
load_dotenv('.env')

oracle_config = {
    'host':  os.getenv('ORACLE_HOST'),
    'user': os.getenv('ORACLE_USER'),
    'pwd': os.getenv('ORACLE_PASS'),
    'port': os.getenv('ORACLE_PORT'),
    'ip': os.getenv('ORACLE_IP'),
    'service_name': os.getenv('ORACLE_SERVICE_NAME')
}

In [3]:
from oracle_helper import OracleHelper

In [4]:
db = OracleHelper(config=oracle_config)

In [5]:
df = pd.read_csv("data/Booking.csv")
df

,BookingsID,InvoiceNumber,DateBooked,DateOfDeparture,FirstName,LastName,Email,Phone,City,State,Country,NumberOfGuests,TotalRevenues,Discount,DiscountPercentage,TotalInvoiceNoDiscounts
0,fc178819-0509-ed11-80ee-00155d027932,2663,2022-06-03T00:00:00,2023-09-26T08:00:00,Jill,Cook,J.Cook@latrobe.edu.au,+61410754549,Melbourne,3078,Australia,2,19600.0,NaN,NaN,NaN
1,b0ccf8fe-50dc-ec11-80ea-00155d026f1f,2643,2022-05-25T12:34:00,2023-02-11T08:00:00,Kathryn,Baddock,robinkate@xtra.co.nz,+64272838111,Warkworth,NaN,NZ,2,17046.0,NaN,NaN,NaN
2,56a85655-0409-ed11-80ee-00155d027932,2665,2022-06-03T00:00:00,2023-09-26T08:00:00,Jodie,Hemmett,jlhemmett@bigpond.com,+61408894547,North Epping,2121,Australia,2,20800.0,NaN,NaN,NaN
3,ce268ac0-f538-ec11-80e6-00155d02748d,2245,2021-10-29T15:20:00,2022-08-16T08:00:00,Nicolas,Van Haaren,Nicolasvanhaaren@gmail.com,+32475230459,Brussegem,NaN,Belgium,2,22400.0,NaN,NaN,NaN
4,aa34e421-79d0-ec11-80f5-00155d026609,2612,2022-05-06T00:00:00,2022-08-19T08:00:00,Julija,Zandersone,julija.zandersone@gmail.com,+37126557110,NaN,NaN,Latvia,1,12750.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,7c4813c2-6ee6-eb11-80e9-00155d02a56a,2037,2021-07-16T12:46:00,2021-10-30T08:00:00,Logan,Thomas Clausen,LOGAN.THOMASCLAUSEN@GMAIL.COM,5414196733,Bend,OR,United States,5,62124.0,-26662.0,0.3002,88786.0
729,d684b676-46d5-eb11-80e9-00155d02a56a,1989,2021-06-24T16:45:00,2022-03-29T08:00:00,James,Corsellis,claudiashaw@marwyn.com,447771806685,London,NaN,United Kingdom,6,61420.0,NaN,NaN,NaN
730,dd4adc63-e502-ed11-80ed-00155d026f1f,2743,2022-07-13T14:51:00,2023-03-25T08:00:00,Robert,Gibbs,robgibbs@outlook.com,+447510037011,Brackley,NaN,United Kingdom,7,64262.0,-2240.0,0.0336,66502.0
731,e61e9344-3026-ec11-80ea-00155d027399,2205,2021-10-05T17:26:00,2021-12-07T08:00:00,Nigel,Knight,Nigeljknight@yahoo.com,447887847002,NaN,NaN,United Kingdom,2,19420.0,-5200.0,0.2112,24620.0


In [6]:
db.insert(df=df, table_name="ADMIN.booking", date_col=["DateBooked", "DateOfDeparture"], date_format=["%Y-%m-%dT%H:%M:%S", "%Y-%m-%dT%H:%M:%S"])

CREATE TABLE admin.booking (bookingsid varchar2(2000),invoicenumber number(30),datebooked timestamp,dateofdeparture timestamp,firstname varchar2(2000),lastname varchar2(2000),email varchar2(2000),phone varchar2(2000),city varchar2(2000),state varchar2(2000),country varchar2(2000),numberofguests number(30),totalrevenues float(30),discount float(30),discountpercentage float(30),totalinvoicenodiscounts float(30) )


100%|██████████| 733/733 [00:00<00:00, 4969.03it/s]


In [8]:
db.drop_table("ADMIN.booking")

BEGIN
                   EXECUTE IMMEDIATE 'DROP TABLE ADMIN.booking';
                EXCEPTION
                   WHEN OTHERS THEN
                      IF SQLCODE != -942 THEN
                         RAISE;
                      END IF;
                END;
excecute done


(True, None)

In [13]:
query = """
create view ADMIN.VW_BOOKING as 
select CITY, count(distinct INVOICENUMBER) AS TotalInvoice
from ADMIN.BOOKING 
GROUP by CITY
"""

db.execute_query(query=query)


create view ADMIN.VW_BOOKING as 
select CITY, count(distinct INVOICENUMBER) AS TotalInvoice
from ADMIN.BOOKING 
GROUP by CITY

excecute done


(True, None)

In [14]:
query = """
select COUNTRY, count(distinct INVOICENUMBER) AS TotalInvoice
from ADMIN.BOOKING 
GROUP by COUNTRY
"""

df_query = db.execute_to_df(query=query)
df_query

c:\Users\ADMIN\OneDrive - thoimmo\Documents\training\python-for-data-engineering\5. Project\0.ETL-data-to-Oracle\oracle_helper.py:96: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,COUNTRY,TOTALINVOICE
0,NL,1
1,United States,221
2,Saudi Arabia,1
3,Israel,1
4,Netherlands,1
5,Spain,1
6,Latvia,1
7,None,21
8,SG,1
9,Mexico,2


In [16]:
db.close()

DatabaseError: ORA-24422: error occurred while trying to destroy the Session Pool